In [1]:
import json

def parse_jsonl(file_path):
    data = []
    with open(file_path, 'r') as file:
        for line in file:
            data.append(json.loads(line))
    return data

path = "data/LongDocURL_public.jsonl"
data = parse_jsonl(path)


In [3]:
data[0].keys()

dict_keys(['question_id', 'doc_no', 'total_pages', 'start_end_idx', 'question_type', 'question', 'answer', 'detailed_evidences', 'evidence_pages', 'evidence_sources', 'answer_format', 'task_tag', 'images', 'pdf_path'])

In [8]:
data[1]

{'question_id': 'topic2title_4125651_5_9_3',
 'doc_no': '4125651',
 'total_pages': 104,
 'start_end_idx': [5, 9],
 'question_type': 'topic2title',
 'question': 'Which titles would provide insights into the importance of light efficiency and quality metrics in lighting products?\nSelect titles from the doc that best answer the question, do not alter or analyze the titles themselves.',
 'answer': ['Uiterst hoge lichtefficientie (tot meer dan 100Lm/W)',
  'dimmable2.200KI 2.600K CRI95',
  'ambient-dimmable2.000K·2.900KCRI95'],
 'detailed_evidences': '',
 'evidence_pages': [5, 9],
 'evidence_sources': ['Layout'],
 'answer_format': 'List',
 'task_tag': 'Locating',
 'images': ['/data/oss_bucket_0/achao.dc/public_datasets/pdf_pngs/4000-4999/4125/4125651_0.png',
  '/data/oss_bucket_0/achao.dc/public_datasets/pdf_pngs/4000-4999/4125/4125651_1.png',
  '/data/oss_bucket_0/achao.dc/public_datasets/pdf_pngs/4000-4999/4125/4125651_2.png',
  '/data/oss_bucket_0/achao.dc/public_datasets/pdf_pngs/4000-